# Lab 3 : Feature selection

In [ ]:
import os
import cv2
import random
import numpy as np 
import pandas as pd 
import seaborn as sns

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
from sklearn.preprocessing import StandardScaler #the Standard Scaler : X2 = (X1 - E(X1))/sqrt(Var(X1))
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB


# To plot pretty figures
%matplotlib inline
import matplotlib as mlp
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

#!pip install pymrmr
!pip install mrmr_selection

## Data Loading 

This step loads the images contained in the training dataset.

In [ ]:
def process_image(img, size, grey_scale=True):
    '''
    This function reduces the size of the image and converts 
    it to a grey_scale image.
    '''
    img = cv2.resize(img, size)
    if grey_scale:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)/size[0]
    return img

### Train Dataset

In [ ]:
labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
train_img = [] #contains the images used for training the model
train_labels = [] #label of each image in x_train 
PATH = '../input/brain-tumor-classification-mri/Training'
new_size = (255, 255)

for label in labels:
    img_dir = os.path.join(PATH, label)
    for img_file in os.listdir(img_dir):
        img = cv2.imread(f'{img_dir}/{img_file}')
        img = process_image(img, new_size)
        train_img.append(img)
        train_labels.append(label)
        
train_img = np.stack(train_img)
train_labels = np.stack(train_labels)

print("train_img shape : ", train_img.shape)

### Test Dataset

In [ ]:
labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
test_img = [] #contains the images used for testing the model
test_labels = [] #label of each image in test_img
PATH = '../input/brain-tumor-classification-mri/Testing'
new_size = (255, 255)

for label in labels:
    img_dir = os.path.join(PATH, label)
    for img_file in os.listdir(img_dir):
        img = cv2.imread(f'{img_dir}/{img_file}')
        img = process_image(img, new_size)
        test_img.append(img)
        test_labels.append(label)
        
test_img = np.stack(test_img)
test_labels = np.stack(test_labels)

print("test_img shape : ", test_img.shape)

## Convert Brain Tumor classes to numerical values

In [ ]:
class_map = {
    'no_tumor': 0,
    'glioma_tumor': 1,
    'pituitary_tumor': 2,
    'meningioma_tumor': 3
}

train_labels = np.array([class_map[label] for label in train_labels])
test_labels = np.array([class_map[label] for label in test_labels])

## Reduced dataset creation : 300 samples per class

In [ ]:
list_index = []

glioma_index = np.where(train_labels == 1)[0][0]
list_index.append(glioma_index)

meningioma_index = np.where(train_labels == 2)[0][0]
list_index.append(meningioma_index)

no_tumor_index = np.where(train_labels == 0)[0][0]
list_index.append(no_tumor_index)

pituitary_index = np.where(train_labels == 3)[0][0]
list_index.append(pituitary_index)

x = []
y = []
for ind in list_index:
    x.append(train_img[ind : ind+300])
    y.append(train_labels[ind : ind + 300])


flat_x = np.stack(x)
flat_x = flat_x.reshape((flat_x.shape[0]*flat_x.shape[1], 255*255))

flat_y = np.stack(y)
flat_y = flat_y.reshape((flat_y.shape[0]*flat_y.shape[1]))

print("dataset shape : ", flat_x.shape)

### Feature Ranking

We would use a dictionary to store the rankings of features from each feature selection algortihm

In [ ]:
rankings = dict()

## Feature Selection



### Dimensionality reduction by PCA implementation

PCA enables to extract a specific number of features while conserving an acceptable amount of information (variance) from original features . 
It consists in projecting the original dataset (standardized) in a less dimensional space such that its variance is maximized, given by the formula : $X_{pca} = X_{original}.W$, the number of W columns corresponding to the number of extracted features.
The first component is : 

$$w_{1} = \underset{||w||=1}{argmax}\Bigg\{||Xw||^{2}\Bigg\} = \underset{w}{argmax}\Bigg\{\frac{w^{T}X^{T}Xw}{w^{T}w}\Bigg\} $$


Other W components are obtained by first computing the substraction of $k-1$ PC:
$$\hat{X}_{k} = X -  \sum \limits_{j=1}^{k-1} Xw_{j}w_{j}^{T}$$

And then by computing : 

$$w_{k} = \underset{w}{argmax}\Bigg\{\frac{w^{T}\hat{X}_{k}^{T}\hat{X}_{k}w}{w^{T}w}\Bigg\}$$


In [ ]:
from sklearn.model_selection import train_test_split

flat_train, flat_test, y_train, y_test = train_test_split(flat_x, flat_y, test_size = 0.3, random_state = 42)

In [ ]:
S = StandardScaler()
flat_train = S.fit_transform(flat_train) 

pca = PCA().fit(flat_train)

plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.grid()
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.title('cumvar(PCA components)')


Conserving ~300 features from the images already allows to keep 90% of the original dataset variance. We choose then to perform a PCA with 300 components. So the final samples have 300 features

In [ ]:
# also possible to specify variance explained ration instead of number of components 
# e.g. nb_components = 0.9 means conserving 90% of original variance

pca = PCA(n_components = 300)

pca_train = pca.fit_transform(flat_train)
rankings['pca'] = pca_train
print("dataset final shape : ", pca_train.shape)

### Inverse PCA (300 features) transform and 3D Visualization

In [ ]:
# comparing original samples and their PCA inverser transform
def compare(label = 'no_tumor'):
    #all index of a certain class 
    clas = class_map[label]
    ind = np.where(y_train == clas)[0]
    rand_ind = np.random.choice(ind)

    #PCA inverse transform
    X_inv = pca.inverse_transform(pca_train)

    #original sample
    x_orig = flat_train[rand_ind].reshape(255,255)
    #pca inv transform sample
    x_pca = X_inv[rand_ind].reshape(255,255)

    #plot comparison between original and pca inverse transform
    fig, axs = plt.subplots(1,2, figsize = (15, 5))
    im1 = axs[0].imshow(x_orig, cmap = 'gray')
    axs[0].set_title('Original train sample, class = {}'.format(label))
    plt.colorbar(im1, ax = axs[0])

    im2 = axs[1].imshow(x_pca, cmap = 'gray')
    axs[1].set_title('Inverse PCA transform of same train sample, class = {}'.format(label))
    plt.colorbar(im2, ax = axs[1])
    fig.tight_layout()
    plt.show()
    
for key in class_map.keys():
    compare(label = key)

In [ ]:
scatter_x = pca_train[:, 0]
scatter_y = pca_train[:, 1]
scatter_z = pca_train[:, 2]
group = y_train

cdict = {0: 'red', 1: 'blue', 2: 'green', 3 : "orange"}
identification = {v : k for k,v in class_map.items()}


plt.figure()
ax = plt.axes(projection = '3d')
plt.grid()
for g in np.unique(group):
    ix = np.where(group == g)
    ax.scatter3D(scatter_x[ix], scatter_y[ix], scatter_z[ix], c = cdict[g], label = identification[g], s = 30)
ax.legend(bbox_to_anchor =(1.95,0.75))
plt.xlabel("PCA feature 1")
plt.ylabel("PCA feature 2")
ax.set_zlabel("PCA feature 3")
plt.title("Classes 3D features (by PCA) representation")
plt.show()

The 3D representation in PCA space seems to show poor separability of the samples. We will then test different classical algorithms to check this out. 

## F - Statistic

In [ ]:
from sklearn.feature_selection import f_classif

print(flat_x.shape)
print(flat_y.shape)

f_selected_features = f_classif(flat_train, y_train)[0]
rankings['f'] = f_selected_features

/!\ Plot testing /!\

In [ ]:
plt.imshow(f_selected_features.reshape(255,255), cmap='Blues_r')
plt.colorbar()

## Maximum Relevance Minimum Redundancy (MRMR)

The goal of this method is to substract features that have high relevancy to the labels as well as little redundacy with one another 
[1](http://)

### Reduce image size for MRMR

In [ ]:
new_size = (100,100)
train_img_mrmr = [process_image(img, new_size, grey_scale=False) for img in train_img]

x = []
y = []
for ind in list_index:
    x.append(train_img_mrmr[ind : ind+300])
    y.append(train_labels[ind : ind + 300])


flat_x = np.stack(x)
flat_x = flat_x.reshape((flat_x.shape[0]*flat_x.shape[1], 100*100))

flat_y = np.stack(y)
flat_y = flat_y.reshape((flat_y.shape[0]*flat_y.shape[1]))

print("dataset shape : ", flat_x.shape)

flat_train, flat_test, y_train, y_test = train_test_split(flat_x, flat_y, test_size = 0.3, random_state = 42)

In [ ]:
import mrmr

X = pd.DataFrame(flat_train)
y = pd.Series(y_train)

# select top 10 features using mRMR
from mrmr import mrmr_classif
mrmr_selected_features = mrmr_classif(X=X, y=y, K=100)

rankings['mrmr'] = mrmr_selected_features

In [ ]:
mrmr_selected_features = np.array(mrmr_selected_features)

plt.imshow(mrmr_selected_features.reshape(10,10), cmap='Blues_r')
plt.colorbar()

### Mutual Information

In [ ]:
from sklearn.feature_selection import mutual_info_classif
mi_selected_features = mutual_info_classif(flat_train, y_train)

rankings['mi'] = mi_selected_features

In [ ]:
plt.imshow(mi_selected_features.reshape(128,128), cmap='Blues_r')
plt.colorbar()

## Split Dataset and Test with some classical classifiers

In [ ]:
algos = ['mrmr', 'mi']  # feature selection algorithms
ks = [30, 100, 200, 65025]   

In [ ]:
# Flatten the dataset

X_train = train_img.reshape((train_img.shape[0], 255*255))
y_train = train_labels
X_test = test_img.reshape((test_img.shape[0], 255*255))
y_test = test_labels
print(X_train.shape)
print(X_test.shape)

In [ ]:
# flat_test = S.transform(flat_test)
# pca_test = pca.transform(flat_test)

### Naive Bayes classifier

In [ ]:
NBaccuracy = pd.DataFrame(index = ks, columns = algos)
NB = GaussianNB()

for algo in algos:
    
    for n_feats in ks:
        
        feats = rankings[algo][:n_feats]
        if algo == 'mi':
            print(feats)
        NB.fit(
            X_train[:, feats], y_train,
        ) 
        y_pred = NB.predict(X_test[:,feats])
        cm1 = confusion_matrix(y_test, y_pred)
        nb_accuracy = np.sum(np.diag(cm1))/np.sum(cm1)
        NBaccuracy.loc[ks, algo] = nb_accuracy

        print(f"naive bayes accuracy {n_feats} features: ", nb_accuracy)

### Accuracy Plot
Accuracy of Naive Bayes model trained on different no of features and different feature selection algorithm.

In [ ]:
fig = plt.figure()
ax = plt.axes()
x = ks
y1 = NBaccuracy.pca.values.tolist() 
y2 = NBaccuracy.mrmr.values.tolist()
y3 = NBaccuracy.f.values.tolist()
y4 = NBaccuracy.mi.values.tolist() 

plt.plot(x, y1, '-g', linewidth=2)  
plt.plot(x, y2, '-c', linewidth=2) 
plt.plot(x, y3, '-k', linewidth=2) 
plt.plot(x, y4, '-r', linewidth=2)

plt.xlabel("No of Features")
plt.ylabel("Accuracy");

plt.show()


### SVM (aka GOAT)

In [ ]:
from sklearn import svm

SVM = svm.SVC(kernel = "rbf", decision_function_shape = 'ovr')

SVM.fit(pca_train, y_train)

y_predict = SVM.predict(pca_test)

cm2 = confusion_matrix(y_test, y_predict)
acc2 = np.sum(np.diag(cm2))/np.sum(cm2)
print("SVM accuracy \n", acc2)

SVMaccuracy = pd.DataFrame(index = ks, columns = algos)
SVM = svm.SVC(kernel = "rbf", decision_function_shape = 'ovr')

for algo in algos:
    
    for nfeats in ks:
        
        feats = ranking[algo][:n_feats]

        SVM.fit(
            X_train[:, feats], y_train,
        ) 
        y_pred = SVM.predict(X_test)
        cm1 = confusion_matrix(y_test, y_pred)
        svm_accuracy = np.sum(np.diag(cm1))/np.sum(cm1)
        SVMaccuracy.loc[ks, algo] = svm_accuracy

        print(f"SVM accuracy {nfeats} features: ", svm_accuracy)


### Accuracy Plot for SVM
Accuracy of support vector machine model trained on different no of features and different feature selection algorithm.

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

metrics = ['cityblock', 'cosine', 'minkowski']
neighbors = range(3, 15)
best_acc = 0
best_ney = 0
best_acc = 0
best_cm = np.zeros((4, 4))
best_metric = ''
for metric in metrics :
    for ney in neighbors:
        knn = KNeighborsClassifier(n_neighbors = ney, metric = metric)
        knn.fit(pca_train, y_train)
        y_predict = knn.predict(pca_test)
        cm = confusion_matrix(y_test, y_predict)
        acc = np.sum(np.diag(cm))/np.sum(cm)
        if acc>best_acc:
            best_ney = ney
            best_cm = cm
            best_acc = acc
            best_metric = metric

print("Best metric :", best_metric)
print("Best k neighbors =", best_ney)
print("Best acc =", best_acc)

n_comp = np.arange(3, 404, 50)
acc = []

for n in n_comp:
    pca = PCA(n_components =n)
    pca_train = pca.fit_transform(flat_train)
    
    pca_test = pca.transform(flat_test)
    
    SVM = svm.SVC(kernel = "rbf", decision_function_shape = 'ovr')
    SVM.fit(pca_train, y_train)
    
    y_predict = SVM.predict(pca_test)
    
    cm2 = confusion_matrix(y_test, y_predict)
    acc2 = np.sum(np.diag(cm2))/np.sum(cm2)
    
    acc.append(acc2)

plt.figure()
plt.grid()
plt.plot(n_comp, acc)
plt.xlabel("n_components")
plt.ylabel("SVM accuracy")
plt.title("SVM accuracy in function of number of PCA components kept")
plt.show()

In [ ]:
# new_train = flat_train[:, selected_features]
# new_test = flat_test[:, selected_features]

# SVM = svm.SVC(kernel = "rbf", decision_function_shape = 'ovr')

# SVM.fit(new_train, y_train)

# y_predict = SVM.predict(new_test)

# cm2 = confusion_matrix(y_test, y_predict)
# acc2 = np.sum(np.diag(cm2))/np.sum(cm2)
# print("SVM accuracy \n", acc2)